# Item-Item Recommendations for Non-Users & Streamlit
- Allow users to put in a movie that they liked and output similar movies     
- For a first pass, use a simple tf-idf genre-based recommendation model. Will update with movie profiles from personalized recommendations     
- If same similarity score (aka same genres), recommend highest rated based on weighted average 
    - Generally want diverse recommendations, expose long tail HOWEVER these people are already non-users who are not rating movies, thus want to give credibility to draw them in and get them to watch anything 
    - Nothing to based the diverse recommendations on other than randomness. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Cached Setup

In [321]:
#@st.cache
def cached_functions():
    
    # read in data created in recommendation_data_display.ipynb
    df = pd.read_parquet('recommendation_display.parq')
    
    # recombine genre lists to string for tf-idf
    df['genre_str'] = df.Genres.apply(lambda row: ' '.join(row))
    
    # set up tf-idf and indices
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(df['genre_str'])
    # record list of movie Ids and indices
    movieIds = df['movieId']
    indices = pd.Series(df.index, index=df['movieId'])
    
    return movieIds, indices, tfidf_matrix

In [ ]:
movieIds, indices, tfidf_matrix = cached_functions()

## Recommend movies based on cosine similarity of genre tf-idf
- Input selected_id: movieId 
    - User input: title (with year) from drop down menu. Need year because some titles are duplicate 
    - movieId is unique, so good for the funciton input 
- Find cosine similarity 
- Merge with display data with metadata
- Sort by scores first, weighted average second if same score

In [319]:
def recommend_movies(selected_id, movieIds, indices, tfidf_matrix):
    
    # get index of movies in tfidf matrix
    idx = indices[selected_id]
    tfmat = tfidf_matrix[idx]
    
    # similarity between movie and other movies
    scores = list(linear_kernel(tfmat, tfidf_matrix)[0])

    # scores in order of movieIds, concat back together
    recs = pd.concat([movieIds, pd.Series(scores)], axis = 1, sort = False)

    # combine movieids and scores to get series of recommendations 
    recs = pd.concat([movies, pd.Series(scores)], axis = 1, sort = False)
    recs.columns = ['movieId', 'score']

    # merge with overall data
    recs = pd.merge(recs, df, on = 'movieId')

    # remove original movie
    recs = recs[recs.movieId != selected_id]
    
    # sort by score first, then weighted average second
    # if same similarity score, recommend higher rated movie
    recs = recs.sort_values(['score', 'weighted_avg'], ascending = False)
    
    return recs

In [320]:
recommend_movies(df[df.title_year == 'Toy Story (1995)'].movieId.values[0])

,movieId,score,Title,Year,Description,Duration (Minutes),Genres,Actors,Director(s),Production Company,...,Language(s),Tags,Number of Ratings,Average Rating,weighted_avg,actors_downcased,directors_downcased,title_downcased,title_year,genre_str
4519,4886,1.0,"Monsters, Inc.",2001,"In order to power the city, monsters have to s...",92,"[Children, Adventure, Fantasy, Comedy, Animation]","[John Goodman, Billy Crystal, Mary Gibbs, Stev...","[Pete Docter, David Silverman]",Pixar Animation Studios,...,[English],"[animated, animation, cartoon, childhood, chil...",17279.0,3.845304,66443.0,"[john goodman, billy crystal, mary gibbs, stev...","[pete docter, david silverman]","monsters, inc.","Monsters, Inc. (2001)",Children Adventure Fantasy Comedy Animation
2851,3114,1.0,Toy Story 2,1999,"When Woody is stolen by a toy collector, Buzz ...",92,"[Children, Adventure, Fantasy, Comedy, Animation]","[Tom Hanks, Tim Allen, Joan Cusack, Kelsey Gra...","[John Lasseter, Ash Brannon]",Pixar Animation Studios,...,"[English, Arabic, Spanish]","[adventure, animated, animation, cartoon, chil...",13313.0,3.818448,50835.0,"[tom hanks, tim allen, joan cusack, kelsey gra...","[john lasseter, ash brannon]",toy story 2,Toy Story 2 (1999),Children Adventure Fantasy Comedy Animation
2061,2294,1.0,Antz,1998,A rather neurotic ant tries to break from his ...,83,"[Children, Adventure, Fantasy, Comedy, Animation]","[Woody Allen, Dan Aykroyd, Anne Bancroft, Jane...","[Eric Darnell, Tim Johnson]",DreamWorks,...,[English],"[animated, animation, cartoon, children, compu...",5508.0,3.227033,17774.5,"[woody allen, dan aykroyd, anne bancroft, jane...","[eric darnell, tim johnson]",antz,Antz (1998),Children Adventure Fantasy Comedy Animation
3697,4016,1.0,"Emperor's New Groove, The",2000,Emperor Kuzco (David Spade) is turned into a l...,78,"[Children, Adventure, Fantasy, Comedy, Animation]","[David Spade, John Goodman, Eartha Kitt, Patri...",[Mark Dindal],Walt Disney Feature Animation,...,[English],"[animated, animation, cartoon, comedy, compute...",4116.0,3.628644,14935.5,"[david spade, john goodman, eartha kitt, patri...",[mark dindal],"emperor's new groove, the","Emperor's New Groove, The (2000)",Children Adventure Fantasy Comedy Animation
33921,166461,1.0,Moana,2016,"In Ancient Polynesia, when a terrible curse in...",107,"[Children, Adventure, Fantasy, Comedy, Animation]","[Auli'i Cravalho, Dwayne Johnson, Rachel House...","[Ron Clements, John Musker]",Hurwitz Creative,...,[English],"[adventure, animated, animation, coming of age...",2000.0,3.867500,7735.0,"[auli'i cravalho, dwayne johnson, rachel house...","[ron clements, john musker]",moana,Moana (2016),Children Adventure Fantasy Comedy Animation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,208933,0.0,The Devil's Partner,1961,"An old man sells his soul to the devil, and tu...",73,[Horror],"[Edgar Buchanan, Jean Allison, Richard Crane, ...",[Charles R. Rondeau],Huron Productions Inc.,...,[English],[],0.0,NaN,0.0,"[edgar buchanan, jean allison, richard crane, ...",[charles r. rondeau],the devil's partner,The Devil's Partner (1961),Horror
45003,209069,0.0,Snapshots,2002,An aging hippy and a beautiful young woman mee...,93,"[Drama, Romance]","[Burt Reynolds, Julie Christie, Carmen Chaplin...",[Rudolf van den Berg],Albion Entertainment,...,"[English, Arabic, Dutch]",[],0.0,NaN,0.0,"[burt reynolds, julie christie, carmen chaplin...",[rudolf van den berg],snapshots,Snapshots (2002),Drama Romance
45005,209137,0.0,The Reward's Yours... The Man's Mine,1969,A group of killers seek out wanted gunman El P...,90,[Western],"[Robert Woods, Aldo Berti, Marc Fiorini, Mario...",[Edoardo Mulargia],Filmar Compagnia Cinematografica,...,"[Italian, Spanish]",[],0.0,NaN,0.0,"[robert woods, aldo berti, marc fiorini, mario...",[edoardo mulargia],the reward's yours... the man's mine,The Reward's Yours... The Man's Mine (1969),Western
45006,209145,0.0,Liberté,2019,"1774, a few years before the Fren